In [4]:
import numpy as np
import pandas as pd 
import scanpy as sc
from sklearn import mixture
import os 
import imp
from src import *

In [ ]:
imp.reload(src)

In [2]:
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(8,8))
import seaborn as sns

In [5]:
adata = sc.read_visium('../dataset/goldStandard')
adata.var_names_make_unique()
adata = data_preprocess(adata)

/home/qinxianhan/.conda/envs/sc/lib/python3.11/site-packages/anndata/_core/anndata.py:1899: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/qinxianhan/.conda/envs/sc/lib/python3.11/site-packages/anndata/_core/anndata.py:1899: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [6]:
gmm = mixture.GaussianMixture(n_components=2)
coord = np.array(adata.obs.iloc[:,1:3])
target_list =['CAMK2N1', 'PCP4', 'NEFM', 'NEFH', 'B3GALT2']
target_gene =target_list[1]

In [ ]:
beta =5
labels_list= mrf.mrf_process(adata, gene_id=target_gene, beta = beta,max_iteration=5e4)

In [7]:
beta = 3
labels_list_new= mrf.mrf_with_icmem(adata, gene_id = target_gene, beta = beta, max_iter = 10)

100%|██████████| 10/10 [00:02<00:00,  4.14it/s]

[[-8.70590508e-01  1.00000000e-05]
 [ 9.53788610e-01  3.54916776e-01]]


In [8]:
exp = adata[:,target_gene].X.toarray()
gmm = mixture.GaussianMixture(n_components=2)
gmm.fit(exp)
gmm_labels = gmm.predict(exp)
gmm_labels = label_resort(gmm.means_, gmm_labels)

In [9]:
gmm.means_.reshape(-1), gmm.covariances_.reshape(-1)

(array([-0.87059051,  0.95385807]), array([1.00000000e-06, 3.54804567e-01]))

In [ ]:
adata.obs['old_labels'] = gmm_labels
adata.obs['new_labels'] = labels_list_new
sc.pl.spatial(adata, color=[target_gene,'old_labels','new_labels'])

In [10]:
type(labels_list_new)

numpy.ndarray

In [15]:
np.savetxt('temp/test_labels.csv', labels_list_new, delimiter=',', fmt = '%d')
np.savetxt('temp/test_data.csv',exp, delimiter=',', fmt = '%.4f' )